In [1]:
import core
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import os,getpass
import numpy as np
import glob
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from langchain_chroma import Chroma

In [2]:
link = '../resources/knowledge-base/*'
folders = glob.glob(link)
text_loader_kwargs = {'encoding': 'utf-8'}
documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=1100, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [4]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [5]:
db_name = 'rag_db'
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=OpenAIEmbeddings()).delete_collection()
vector_store = Chroma.from_documents(documents=docs,
                                            embedding=OpenAIEmbeddings(),
                                            persist_directory=db_name)
collection = vector_store._collection

In [6]:
print(f"Vectorstore created with {collection.count()} documents")

Vectorstore created with 105 documents


In [7]:
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [8]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in
          doc_types]

In [9]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [10]:
fig = go.Figure(data=[go.Scatter(
            x=reduced_vectors[:, 0],
            y=reduced_vectors[:, 1],
            mode='markers',
            marker=dict(size=5, color=colors, opacity=0.8),
            text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
            hoverinfo='text'
        )])

fig.update_layout(
            title='2D Chroma Vector Store Visualization',
            scene=dict(xaxis_title='x',yaxis_title='y'),
            width=800,
            height=600,
            margin=dict(r=20, b=10, l=10, t=40)
        )
fig.show()

In [11]:
tsne_3d = TSNE(n_components=3, random_state=42)
reduced_vectors_3d = tsne_3d.fit_transform(vectors)

In [12]:
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors_3d[:, 0],
    y=reduced_vectors_3d[:, 1],
    z=reduced_vectors_3d[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()